In [ ]:
from ultralytics import YOLO
import os,cv2
import argparse

from tracker.ucmc import UCMCTrack
from detector.mapper import Mapper
import numpy as np


# 定义一个Detection类，包含id,bb_left,bb_top,bb_width,bb_height,conf,det_class
class Detection:

    def __init__(self, id, bb_left = 0, bb_top = 0, bb_width = 0, bb_height = 0, conf = 0, det_class = 0):
        self.id = id
        self.bb_left = bb_left
        self.bb_top = bb_top
        self.bb_width = bb_width
        self.bb_height = bb_height
        self.conf = conf
        self.det_class = det_class
        self.track_id = 0
        self.y = np.zeros((2, 1))
        self.R = np.eye(4)


    def __str__(self):
        return 'd{}, bb_box:[{},{},{},{}], conf={:.2f}, class{}, uv:[{:.0f},{:.0f}], mapped to:[{:.1f},{:.1f}]'.format(
            self.id, self.bb_left, self.bb_top, self.bb_width, self.bb_height, self.conf, self.det_class,
            self.bb_left+self.bb_width/2,self.bb_top+self.bb_height,self.y[0,0],self.y[1,0])

    def __repr__(self):
        return self.__str__()


# Detector类，用于从Yolo检测器获取目标检测的结果
class Detector:
    def __init__(self):
        self.seq_length = 0
        self.gmc = None

    def load(self,cam_para_file):
        self.mapper = Mapper(cam_para_file,"MOT17")
        self.model = YOLO('pretrained/yolov8x.pt')

    def get_dets(self, img,conf_thresh = 0,det_classes = [0]):
        
        dets = []

        # 将帧从 BGR 转换为 RGB（因为 OpenCV 使用 BGR 格式）  
        frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  

        # 使用 RTDETR 进行推理  
        results = self.model(frame,imgsz = 1088)

        det_id = 0
        for box in results[0].boxes:
            conf = box.conf.cpu().numpy()[0]
            bbox = box.xyxy.cpu().numpy()[0]
            cls_id  = box.cls.cpu().numpy()[0]
            w = bbox[2] - bbox[0]
            h = bbox[3] - bbox[1]
            if w <= 10 and h <= 10 or cls_id not in det_classes or conf <= conf_thresh:
                continue

            # 新建一个Detection对象
            det = Detection(det_id)
            det.bb_left = bbox[0]
            det.bb_top = bbox[1]
            det.bb_width = w
            det.bb_height = h
            det.conf = conf
            det.det_class = cls_id
            det.y,det.R = self.mapper.mapto([det.bb_left,det.bb_top,det.bb_width,det.bb_height])
            det_id += 1

            dets.append(det)

        return dets
    

def main(args):

    class_list = [2,5,7]

    cap = cv2.VideoCapture(args.video)

    # 获取视频的 fps
    fps = cap.get(cv2.CAP_PROP_FPS)

    # 获取视频的宽度和高度
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    video_out = cv2.VideoWriter('output/output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))  

    # 打开一个cv的窗口，指定高度和宽度
    cv2.namedWindow("demo", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("demo", width, height)

    detector = Detector()
    detector.load(args.cam_para)

    
    tracker = UCMCTrack(args.a, args.a, args.wx, args.wy, args.vmax, args.cdt, fps, "MOT", args.high_score,False,None)

    # 循环读取视频帧
    frame_id = 1
    while True:
        ret, frame_img = cap.read()
        if not ret:  
            break
    
        dets = detector.get_dets(frame_img,args.conf_thresh,class_list)
        tracker.update(dets,frame_id)

        for det in dets:
            # 画出检测框
            if det.track_id > 0:
                cv2.rectangle(frame_img, (int(det.bb_left), int(det.bb_top)), (int(det.bb_left+det.bb_width), int(det.bb_top+det.bb_height)), (0, 255, 0), 2)
                # 画出检测框的id
                cv2.putText(frame_img, str(det.track_id), (int(det.bb_left), int(det.bb_top)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        frame_id += 1


        # 显示当前帧
        cv2.imshow("demo", frame_img)
        cv2.waitKey(1)

        video_out.write(frame_img)
    
    cap.release()
    video_out.release()
    cv2.destroyAllWindows()



parser = argparse.ArgumentParser(description='Process some arguments.')
parser.add_argument('--video', type=str, default = "demo/demo.mp4", help='video file name')
parser.add_argument('--cam_para', type=str, default = "demo/cam_para.txt", help='camera parameter file name')
parser.add_argument('--wx', type=float, default=5, help='wx')
parser.add_argument('--wy', type=float, default=5, help='wy')
parser.add_argument('--vmax', type=float, default=10, help='vmax')
parser.add_argument('--a', type=float, default=100.0, help='assignment threshold')
parser.add_argument('--cdt', type=float, default=10.0, help='coasted deletion time')
parser.add_argument('--high_score', type=float, default=0.5, help='high score threshold')
parser.add_argument('--conf_thresh', type=float, default=0.01, help='detection confidence threshold')
args = parser.parse_args()

main(args)


# working plot without canvas

In [ ]:
import numpy as np
import cv2
import json
import os
import matplotlib.pyplot as plt
from ultralytics import YOLO
from tracker.ucmc import UCMCTrack
from detector.mapper import Mapper
from collections import defaultdict
import argparse

# 定义一个Detection类
class Detection:
    def __init__(self, id, bb_left=0, bb_top=0, bb_width=0, bb_height=0, conf=0, det_class=0):
        self.id = id
        self.bb_left = bb_left
        self.bb_top = bb_top
        self.bb_width = bb_width
        self.bb_height = bb_height
        self.conf = conf
        self.det_class = det_class
        self.track_id = 0
        self.y = np.zeros((2, 1))
        self.R = np.eye(4)

    def __str__(self):
        return f'd{self.id}, bb_box:[{self.bb_left},{self.bb_top},{self.bb_width},{self.bb_height}], conf={self.conf:.2f}, class{self.det_class}, uv:[{self.bb_left+self.bb_width/2},{self.bb_top+self.bb_height}], mapped to:[{self.y[0,0]},{self.y[1,0]}]'

    def __repr__(self):
        return self.__str__()

# Detector类
class Detector:
    def __init__(self):
        self.seq_length = 0
        self.gmc = None

    def load(self, cam_para_file, yolo_version):
        self.mapper = Mapper(cam_para_file, "MOT17")
        self.model = YOLO(f'pretrained/{yolo_version}.pt')

    def get_dets(self, img, conf_thresh=0, det_classes=[0]):
        dets = []
        frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = self.model(frame, imgsz=1088)
        det_id = 0
        for box in results[0].boxes:
            conf = box.conf.cpu().numpy()[0]
            bbox = box.xyxy.cpu().numpy()[0]
            cls_id = box.cls.cpu().numpy()[0]
            w = bbox[2] - bbox[0]
            h = bbox[3] - bbox[1]
            if w <= 10 and h <= 10 or cls_id not in det_classes or conf <= conf_thresh:
                continue

            det = Detection(det_id)
            det.bb_left = bbox[0]
            det.bb_top = bbox[1]
            det.bb_width = w
            det.bb_height = h
            det.conf = conf
            det.det_class = cls_id
            det.y, det.R = self.mapper.mapto([det.bb_left, det.bb_top, det.bb_width, det.bb_height])
            det_id += 1
            dets.append(det)

        return dets

# Function to apply the ROI mask with green overlay
def apply_green_roi_mask(frame, points):
    overlay = frame.copy()
    cv2.fillPoly(overlay, [points], (0, 255, 0))  # Green color
    alpha = 0.3  # Transparency
    cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
    return frame

# Set up ROI points
roi_points = np.array([
    [28, 1046], [109, 977], [269, 846], [426, 715], [507, 614],
    [570, 520], [521, 456], [427, 411], [308, 392], [182, 372],
    [39, 350], [196, 251], [338, 273], [518, 299], [663, 315],
    [805, 319], [920, 292], [1045, 236], [1099, 195], [1155, 154],
    [1243, 103], [1311, 114], [1256, 176], [1205, 249], [1198, 346],
    [1286, 433], [1454, 500], [1634, 555], [1786, 595], [1902, 621],
    [1888, 797], [1665, 738], [1408, 684], [1184, 650], [1012, 680],
    [856, 774], [739, 921], [659, 1047]
], np.int32)

# Main function to run video and plot real-time trajectories
def main(args):
    yolo_version = "yolov8x"
    detector = Detector()
    detector.load(args.cam_para, yolo_version)
    tracker = UCMCTrack(args.a, args.a, args.wx, args.wy, args.vmax, args.cdt, args.fps, "MOT", args.high_score, False, None)
    unique_car_ids = set()
    trajectory_data = defaultdict(lambda: {'coords': [], 'class_id': None})
    
    cap = cv2.VideoCapture(args.video)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Set up Matplotlib figure for trajectories
    fig, ax = plt.subplots()
    plt.ion()  # Interactive mode for real-time plotting

    def update_trajectory_graph():
        ax.clear()
        ax.set_title("Trajectories of Tracked Vehicles")
        ax.set_xlabel("X Coordinate")
        ax.set_ylabel("Y Coordinate")
        for track_id, data in trajectory_data.items():
            if data['coords']:
                x_vals, y_vals = zip(*data['coords'])
                class_id = data['class_id']
                label = "Car" if class_id == 2 else f"Class {class_id}"
                ax.plot(x_vals, y_vals, marker="o", label=f'Track {track_id} ({label})')
        ax.legend(loc="upper right")
        plt.draw()
        plt.pause(0.001)

    # Process each frame
    frame_id = 1
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = apply_green_roi_mask(frame, roi_points)

        dets = detector.get_dets(frame, args.conf_thresh, [2, 5, 7])
        tracker.update(dets, frame_id)

        for det in dets:
            if det.track_id > 0:
                unique_car_ids.add(det.track_id)
                trajectory_data[det.track_id]['coords'].append((det.y[0, 0], det.y[1, 0]))
                trajectory_data[det.track_id]['class_id'] = det.det_class
                cv2.rectangle(frame, (int(det.bb_left), int(det.bb_top)),
                              (int(det.bb_left + det.bb_width), int(det.bb_top + det.bb_height)),
                              (0, 255, 0), 2)
                cv2.putText(frame, str(det.track_id), (int(det.bb_left), int(det.bb_top)),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Display car count
        cv2.putText(frame, f"Cars: {len(unique_car_ids)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        cv2.imshow("Real-Time Tracking", frame)
        update_trajectory_graph()

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_id += 1

    cap.release()
    cv2.destroyAllWindows()
    plt.close()

# Argument parser setup
parser = argparse.ArgumentParser(description='Real-time Tracking and Trajectory Plotting')
parser.add_argument('--video', type=str, default="C:/research/new30s.mp4", help='video file name')
parser.add_argument('--cam_para', type=str, default="C:/research/UCMCTrack/demo/cam_para.txt", help='camera parameter file name')
parser.add_argument('--wx', type=float, default=5, help='wx')
parser.add_argument('--wy', type=float, default=5, help='wy')
parser.add_argument('--vmax', type=float, default=10, help='vmax')
parser.add_argument('--a', type=float, default=150.0, help='assignment threshold')
parser.add_argument('--cdt', type=float, default=15.0, help='coasted deletion time')
parser.add_argument('--high_score', type=float, default=0.4, help='high score threshold')
parser.add_argument('--conf_thresh', type=float, default=0.001, help='detection confidence threshold')
parser.add_argument('--fps', type=float, default=30, help='frames per second')
args = parser.parse_args()

# Run the main function
main(args)


# wrong shape

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO
from tracker.ucmc import UCMCTrack
from detector.mapper import Mapper
from collections import defaultdict
import argparse

# Define Detection class
class Detection:
    def __init__(self, id, bb_left=0, bb_top=0, bb_width=0, bb_height=0, conf=0, det_class=0):
        self.id = id
        self.bb_left = bb_left
        self.bb_top = bb_top
        self.bb_width = bb_width
        self.bb_height = bb_height
        self.conf = conf
        self.det_class = det_class
        self.track_id = 0
        self.y = np.zeros((2, 1))
        self.R = np.eye(4)

# Define Detector class
class Detector:
    def __init__(self):
        self.seq_length = 0
        self.gmc = None

    def load(self, cam_para_file, yolo_version):
        self.mapper = Mapper(cam_para_file, "MOT17")
        self.model = YOLO(f'pretrained/{yolo_version}.pt')

    def get_dets(self, img, conf_thresh=0, det_classes=[0]):
        dets = []
        frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = self.model(frame, imgsz=1088)
        det_id = 0
        for box in results[0].boxes:
            conf = box.conf.cpu().numpy()[0]
            bbox = box.xyxy.cpu().numpy()[0]
            cls_id = box.cls.cpu().numpy()[0]
            w = bbox[2] - bbox[0]
            h = bbox[3] - bbox[1]
            if w <= 10 and h <= 10 or cls_id not in det_classes or conf <= conf_thresh:
                continue

            det = Detection(det_id)
            det.bb_left = bbox[0]
            det.bb_top = bbox[1]
            det.bb_width = w
            det.bb_height = h
            det.conf = conf
            det.det_class = cls_id
            det.y, det.R = self.mapper.mapto([det.bb_left, det.bb_top, det.bb_width, det.bb_height])
            det_id += 1
            dets.append(det)

        return dets

# Main function to run video and plot real-time trajectories
def main(args):
    yolo_version = "yolov8x"
    detector = Detector()
    detector.load(args.cam_para, yolo_version)
    tracker = UCMCTrack(args.a, args.a, args.wx, args.wy, args.vmax, args.cdt, args.fps, "MOT", args.high_score, False, None)
    unique_car_ids = set()
    trajectory_data = defaultdict(lambda: {'coords': [], 'class_id': None})

    cap = cv2.VideoCapture(args.video)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Initialize a white canvas as the main display
    canvas_width = width + 400  # Additional width for the trajectory and car count
    canvas_height = height
    car_count_position = (width + 20, canvas_height - 30)  # Bottom-right corner of the canvas

    # Set up Matplotlib figure for the trajectory plot
    fig, ax = plt.subplots()
    plt.ion()  # Interactive mode for real-time plotting

    def update_trajectory_graph():
        ax.clear()
        ax.set_title("Trajectories of Tracked Vehicles")
        ax.set_xlabel("X Coordinate")
        ax.set_ylabel("Y Coordinate")
        for track_id, data in trajectory_data.items():
            if data['coords']:
                x_vals, y_vals = zip(*data['coords'])
                class_id = data['class_id']
                label = "Car" if class_id == 2 else f"Class {class_id}"
                ax.plot(x_vals, y_vals, marker="o", label=f'Track {track_id} ({label})')
        ax.legend(loc="upper right")
        fig.canvas.draw()
        fig_image = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
        fig_image = fig_image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        return fig_image

    frame_id = 1
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        dets = detector.get_dets(frame, args.conf_thresh, [2, 5, 7])
        tracker.update(dets, frame_id)

        # Update the unique car count and trajectory data
        for det in dets:
            if det.track_id > 0:
                unique_car_ids.add(det.track_id)
                trajectory_data[det.track_id]['coords'].append((det.y[0, 0], det.y[1, 0]))
                trajectory_data[det.track_id]['class_id'] = det.det_class
                cv2.rectangle(frame, (int(det.bb_left), int(det.bb_top)),
                              (int(det.bb_left + det.bb_width), int(det.bb_top + det.bb_height)),
                              (0, 255, 0), 2)
                cv2.putText(frame, str(det.track_id), (int(det.bb_left), int(det.bb_top)),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Create a white canvas and place the video frame on it
        canvas = np.ones((canvas_height, canvas_width, 3), dtype=np.uint8) * 255
        canvas[0:height, 0:width] = frame

        # Get the updated trajectory plot and place it in the top-right corner
        trajectory_img = update_trajectory_graph()
        traj_height, traj_width, _ = trajectory_img.shape
        canvas[10:10+traj_height, width+10:width+10+traj_width] = trajectory_img

        # Display car count in the bottom-right corner
        cv2.putText(canvas, f"Cars: {len(unique_car_ids)}", car_count_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

        # Display the canvas
        cv2.imshow("Real-Time Tracking and Analysis", canvas)

        # Quit if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_id += 1

    cap.release()
    cv2.destroyAllWindows()
    plt.close()

# Argument parser setup
parser = argparse.ArgumentParser(description='Real-time Tracking and Trajectory Plotting')
parser.add_argument('--video', type=str, default="C:/research/new30s.mp4", help='video file name')
parser.add_argument('--cam_para', type=str, default="C:/research/UCMCTrack/demo/cam_para.txt", help='camera parameter file name')
parser.add_argument('--wx', type=float, default=5, help='wx')
parser.add_argument('--wy', type=float, default=5, help='wy')
parser.add_argument('--vmax', type=float, default=10, help='vmax')
parser.add_argument('--a', type=float, default=150.0, help='assignment threshold')
parser.add_argument('--cdt', type=float, default=15.0, help='coasted deletion time')
parser.add_argument('--high_score', type=float, default=0.4, help='high score threshold')
parser.add_argument('--conf_thresh', type=float, default=0.001, help='detection confidence threshold')
parser.add_argument('--fps', type=float, default=30, help='frames per second')
args = parser.parse_args()

# Run the main function
main(args)


# working version


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO
from tracker.ucmc import UCMCTrack
from detector.mapper import Mapper
from collections import defaultdict
import argparse

# Define Detection class
class Detection:
    def __init__(self, id, bb_left=0, bb_top=0, bb_width=0, bb_height=0, conf=0, det_class=0):
        self.id = id
        self.bb_left = bb_left
        self.bb_top = bb_top
        self.bb_width = bb_width
        self.bb_height = bb_height
        self.conf = conf
        self.det_class = det_class
        self.track_id = 0
        self.y = np.zeros((2, 1))
        self.R = np.eye(4)

# Define Detector class
class Detector:
    def __init__(self):
        self.seq_length = 0
        self.gmc = None

    def load(self, cam_para_file, yolo_version):
        self.mapper = Mapper(cam_para_file, "MOT17")
        self.model = YOLO(f'pretrained/{yolo_version}.pt')

    def get_dets(self, img, conf_thresh=0, det_classes=[0]):
        dets = []
        frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = self.model(frame, imgsz=1088)
        det_id = 0
        for box in results[0].boxes:
            conf = box.conf.cpu().numpy()[0]
            bbox = box.xyxy.cpu().numpy()[0]
            cls_id = box.cls.cpu().numpy()[0]
            w = bbox[2] - bbox[0]
            h = bbox[3] - bbox[1]
            if w <= 10 and h <= 10 or cls_id not in det_classes or conf <= conf_thresh:
                continue

            det = Detection(det_id)
            det.bb_left = bbox[0]
            det.bb_top = bbox[1]
            det.bb_width = w
            det.bb_height = h
            det.conf = conf
            det.det_class = cls_id
            det.y, det.R = self.mapper.mapto([det.bb_left, det.bb_top, det.bb_width, det.bb_height])
            det_id += 1
            dets.append(det)

        return dets

# Main function to run video and plot real-time trajectories
def main(args):
    yolo_version = "yolov8x"
    detector = Detector()
    detector.load(args.cam_para, yolo_version)
    tracker = UCMCTrack(args.a, args.a, args.wx, args.wy, args.vmax, args.cdt, args.fps, "MOT", args.high_score, False, None)
    unique_car_ids = set()
    trajectory_data = defaultdict(lambda: {'coords': [], 'class_id': None})

    cap = cv2.VideoCapture(args.video)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define scaled down dimensions for the video frame
    video_display_width = width // 2  # Adjust this value if needed
    video_display_height = height // 2
    canvas_width = video_display_width + 400  # Extra space for trajectory plot
    canvas_height = video_display_height + 200  # Extra space for car count display

    # Set up position for car count display
    car_count_position = (video_display_width + 20, canvas_height - 30)  # Bottom-right corner

    # Set up Matplotlib figure for the trajectory plot
    fig, ax = plt.subplots()
    plt.ion()  # Interactive mode for real-time plotting

    def update_trajectory_graph():
        ax.clear()
        ax.set_title("Trajectories of Tracked Vehicles")
        ax.set_xlabel("X Coordinate")
        ax.set_ylabel("Y Coordinate")
        if not trajectory_data:
            print("No trajectory data to display.")
        for track_id, data in trajectory_data.items():
            if data['coords']:
                x_vals, y_vals = zip(*data['coords'])
                class_id = data['class_id']
                label = "Car" if class_id == 2 else f"Class {class_id}"
                ax.plot(x_vals, y_vals, marker="o", label=f'Track {track_id} ({label})')
        if trajectory_data:
            ax.legend(loc="upper right")
        fig.canvas.draw()
        fig_image = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
        fig_image = fig_image.reshape(fig.canvas.get_width_height()[::-1] + (4,))
        fig_image = cv2.cvtColor(fig_image, cv2.COLOR_RGBA2RGB)  # Convert RGBA to RGB
        return fig_image

    frame_id = 1
    while True:
        ret, frame = cap.read()
        if not ret:
            print("No more frames to read.")
            break

        # Scale down the video frame for display
        frame_resized = cv2.resize(frame, (video_display_width, video_display_height))

        dets = detector.get_dets(frame, args.conf_thresh, [2, 5, 7])
        tracker.update(dets, frame_id)

        # Update the unique car count and trajectory data
        for det in dets:
            if det.track_id > 0:
                unique_car_ids.add(det.track_id)
                trajectory_data[det.track_id]['coords'].append((det.y[0, 0], det.y[1, 0]))
                trajectory_data[det.track_id]['class_id'] = det.det_class
                cv2.rectangle(frame_resized, (int(det.bb_left // 2), int(det.bb_top // 2)),
                              (int((det.bb_left + det.bb_width) // 2), int((det.bb_top + det.bb_height) // 2)),
                              (0, 255, 0), 2)
                cv2.putText(frame_resized, str(det.track_id), (int(det.bb_left // 2), int(det.bb_top // 2)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        # Create a white canvas and place the video frame on it
        canvas = np.ones((canvas_height, canvas_width, 3), dtype=np.uint8) * 255
        canvas[0:video_display_height, 0:video_display_width] = frame_resized

        # Get the updated trajectory plot and place it in the top-right corner
        trajectory_img = update_trajectory_graph()
        traj_height, traj_width, _ = trajectory_img.shape
        resized_traj_img = cv2.resize(trajectory_img, (390, int(traj_height * (390 / traj_width))))  # Resize proportionally
        canvas[10:10+resized_traj_img.shape[0], video_display_width+10:video_display_width+10+resized_traj_img.shape[1]] = resized_traj_img

        # Display car count in the bottom-right corner
        cv2.putText(canvas, f"Cars: {len(unique_car_ids)}", car_count_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

        # Display the canvas
        cv2.imshow("Real-Time Tracking and Analysis", canvas)

        # Quit if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_id += 1
        # Debugging output
        print(f"Frame {frame_id}, Cars Count: {len(unique_car_ids)}, Trajectories: {len(trajectory_data)}")

    cap.release()
    cv2.destroyAllWindows()
    plt.close()

# Argument parser setup
parser = argparse.ArgumentParser(description='Real-time Tracking and Trajectory Plotting')
parser.add_argument('--video', type=str, default="C:/research/new30s.mp4", help='video file name')
parser.add_argument('--cam_para', type=str, default="C:/research/UCMCTrack/demo/cam_para.txt", help='camera parameter file name')
parser.add_argument('--wx', type=float, default=5, help='wx')
parser.add_argument('--wy', type=float, default=5, help='wy')
parser.add_argument('--vmax', type=float, default=10, help='vmax')
parser.add_argument('--a', type=float, default=150.0, help='assignment threshold')
parser.add_argument('--cdt', type=float, default=15.0, help='coasted deletion time')
parser.add_argument('--high_score', type=float, default=0.4, help='high score threshold')
parser.add_argument('--conf_thresh', type=float, default=0.001, help='detection confidence threshold')
parser.add_argument('--fps', type=float, default=30, help='frames per second')
args = parser.parse_args()

# Run the main function
main(args)


In [ ]:
# # Main function to run video and plot real-time trajectories
# def main(args):
#     yolo_version = "yolov8x"
#     detector = Detector()
#     detector.load(args.cam_para, yolo_version)
#     tracker = UCMCTrack(args.a, args.a, args.wx, args.wy, args.vmax, args.cdt, args.fps, "MOT", args.high_score, False, None)
#     unique_car_ids = set()
#     trajectory_data = defaultdict(lambda: {'coords': [], 'class_id': None})

#     cap = cv2.VideoCapture(args.video)
#     fps = cap.get(cv2.CAP_PROP_FPS)
#     width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#     height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

#     # Define scaled down dimensions for the video frame
#     video_display_width = width // 2  # Adjust this value if needed
#     video_display_height = height // 2
#     canvas_width = video_display_width + 400  # Extra space for trajectory plot
#     canvas_height = video_display_height + 10#+ 200  # Extra space for car count display

#     # Set up position for car count display
#     car_count_position = (video_display_width + 10, canvas_height - 20)  # Bottom-right corner

#     # Set up Matplotlib figure for the trajectory plot
#     fig, ax = plt.subplots()
#     plt.ion()  # Interactive mode for real-time plotting

#     def update_trajectory_graph():
#         ax.clear()
#         ax.set_title("Trajectories of Tracked Vehicles")
#         ax.set_xlabel("X Coordinate")
#         ax.set_ylabel("Y Coordinate")
#         if not trajectory_data:
#             print("No trajectory data to display.")
#         for track_id, data in trajectory_data.items():
#             if data['coords']:
#                 x_vals, y_vals = zip(*data['coords'])
#                 class_id = data['class_id']
#                 label = "Car" if class_id == 2 else f"Class {class_id}"
#                 ax.plot(x_vals, y_vals, marker="o", label=f'Track {track_id} ({label})')
#         if trajectory_data:
#             ax.legend(loc="upper right")
#         fig.canvas.draw()
#         fig_image = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
#         fig_image = fig_image.reshape(fig.canvas.get_width_height()[::-1] + (4,))
#         fig_image = cv2.cvtColor(fig_image, cv2.COLOR_RGBA2RGB)  # Convert RGBA to RGB
#         return fig_image

#     frame_id = 1
#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             print("No more frames to read.")
#             break

#         # Scale down the video frame for display
#         frame_resized = cv2.resize(frame, (video_display_width, video_display_height))

#         dets = detector.get_dets(frame, args.conf_thresh, [2, 5, 7])
#         tracker.update(dets, frame_id)

#         # Update the unique car count and trajectory data
#         for det in dets:
#             if det.track_id > 0:
#                 unique_car_ids.add(det.track_id)
#                 trajectory_data[det.track_id]['coords'].append((det.y[0, 0], det.y[1, 0]))
#                 trajectory_data[det.track_id]['class_id'] = det.det_class
#                 cv2.rectangle(frame_resized, (int(det.bb_left // 2), int(det.bb_top // 2)),
#                               (int((det.bb_left + det.bb_width) // 2), int((det.bb_top + det.bb_height) // 2)),
#                               (0, 255, 0), 2)
#                 cv2.putText(frame_resized, str(det.track_id), (int(det.bb_left // 2), int(det.bb_top // 2)),
#                             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

#         # Create a white canvas and place the video frame on it
#         canvas = np.ones((canvas_height, canvas_width, 3), dtype=np.uint8) * 255
#         canvas[0:video_display_height, 0:video_display_width] = frame_resized

#         # Get the updated trajectory plot and place it in the top-right corner
#         trajectory_img = update_trajectory_graph()
#         traj_height, traj_width, _ = trajectory_img.shape
#         resized_traj_img = cv2.resize(trajectory_img, (390, int(traj_height * (390 / traj_width))))  # Resize proportionally
#         canvas[10:10+resized_traj_img.shape[0], video_display_width+10:video_display_width+10+resized_traj_img.shape[1]] = resized_traj_img

#         # Display car count in the bottom-right corner
#         cv2.putText(canvas, f"Cars: {len(unique_car_ids)}", car_count_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

#         # Display the canvas
#         cv2.imshow("Real-Time Tracking and Analysis", canvas)

#         # Quit if 'q' is pressed
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

#         frame_id += 1
#         # Debugging output
#         print(f"Frame {frame_id}, Cars Count: {len(unique_car_ids)}, Trajectories: {len(trajectory_data)}")

#     cap.release()
#     cv2.destroyAllWindows()
#     plt.close()

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO
from tracker.ucmc import UCMCTrack
from detector.mapper import Mapper
from collections import defaultdict
import argparse
# Define the ROI points as a numpy array
roi_points = np.array([
    [28, 1046], [109, 977], [269, 846], [426, 715], [507, 614],
    [570, 520], [521, 456], [427, 411], [308, 392], [182, 372],
    [39, 350], [196, 251], [338, 273], [518, 299], [663, 315],
    [805, 319], [920, 292], [1045, 236], [1099, 195], [1155, 154],
    [1243, 103], [1311, 114], [1256, 176], [1205, 249], [1198, 346],
    [1286, 433], [1454, 500], [1634, 555], [1786, 595], [1902, 621],
    [1888, 797], [1665, 738], [1408, 684], [1184, 650], [1012, 680],
    [856, 774], [739, 921], [659, 1047]
], np.int32)

# Define Detection class
class Detection:
    def __init__(self, id, bb_left=0, bb_top=0, bb_width=0, bb_height=0, conf=0, det_class=0):
        self.id = id
        self.bb_left = bb_left
        self.bb_top = bb_top
        self.bb_width = bb_width
        self.bb_height = bb_height
        self.conf = conf
        self.det_class = det_class
        self.track_id = 0
        self.y = np.zeros((2, 1))
        self.R = np.eye(4)

# Define Detector class
class Detector:
    def __init__(self):
        self.seq_length = 0
        self.gmc = None

    def load(self, cam_para_file, yolo_version):
        self.mapper = Mapper(cam_para_file, "MOT17")
        self.model = YOLO(f'pretrained/{yolo_version}.pt')

    def get_dets(self, img, conf_thresh=0, det_classes=[0]):
        dets = []
        frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = self.model(frame, imgsz=1088)
        det_id = 0
        for box in results[0].boxes:
            conf = box.conf.cpu().numpy()[0]
            bbox = box.xyxy.cpu().numpy()[0]
            cls_id = box.cls.cpu().numpy()[0]
            w = bbox[2] - bbox[0]
            h = bbox[3] - bbox[1]
            if w <= 10 and h <= 10 or cls_id not in det_classes or conf <= conf_thresh:
                continue

            det = Detection(det_id)
            det.bb_left = bbox[0]
            det.bb_top = bbox[1]
            det.bb_width = w
            det.bb_height = h
            det.conf = conf
            det.det_class = cls_id
            det.y, det.R = self.mapper.mapto([det.bb_left, det.bb_top, det.bb_width, det.bb_height])
            det_id += 1
            dets.append(det)

        return dets
    
def apply_green_roi_mask(frame, points):
    overlay = frame.copy()
    cv2.fillPoly(overlay, [points], (0, 255, 0))  # Green color
    alpha = 0.3  # Transparency
    cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
    return frame

def main(args):
    yolo_version = "yolov8x"
    detector = Detector()
    detector.load(args.cam_para, yolo_version)
    tracker = UCMCTrack(args.a, args.a, args.wx, args.wy, args.vmax, args.cdt, args.fps, "MOT", args.high_score, False, None)
    unique_car_ids = set()
    trajectory_data = defaultdict(lambda: {'coords': [], 'class_id': None})

    cap = cv2.VideoCapture(args.video)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define scaled down dimensions for the video frame
    video_display_width = width // 2
    video_display_height = height // 2
    canvas_width = video_display_width + 400
    canvas_height = video_display_height + 10
    car_count_position = (video_display_width + 10, canvas_height - 20)

    fig, ax = plt.subplots()
    plt.ion()

    def update_trajectory_graph():
        ax.clear()
        ax.set_title("Trajectories of Tracked Vehicles")
        ax.set_xlabel("X Coordinate")
        ax.set_ylabel("Y Coordinate")
        for track_id, data in trajectory_data.items():
            if data['coords']:
                x_vals, y_vals = zip(*data['coords'])
                ax.plot(x_vals, y_vals, marker="o", label=f'Track {track_id}')
        fig.canvas.draw()
        fig_image = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
        fig_image = fig_image.reshape(fig.canvas.get_width_height()[::-1] + (4,))
        fig_image = cv2.cvtColor(fig_image, cv2.COLOR_RGBA2RGB)
        return fig_image

    frame_id = 1
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Apply the green ROI mask to the video frame
        frame_resized = cv2.resize(frame, (video_display_width, video_display_height))
        frame_resized = apply_green_roi_mask(frame_resized, roi_points)

        # Process detections and filter within the ROI
        dets = detector.get_dets(frame, args.conf_thresh, [2, 5, 7])
        tracker.update(dets, frame_id)

        for det in dets:
            # Check if detection is inside the ROI
            center = (int(det.bb_left + det.bb_width / 2), int(det.bb_top + det.bb_height / 2))
            if cv2.pointPolygonTest(roi_points, center, False) >= 0:
                unique_car_ids.add(det.track_id)
                trajectory_data[det.track_id]['coords'].append((det.y[0, 0], det.y[1, 0]))
                cv2.rectangle(frame_resized, (int(det.bb_left // 2), int(det.bb_top // 2)),
                              (int((det.bb_left + det.bb_width) // 2), int((det.bb_top + det.bb_height) // 2)),
                              (0, 255, 0), 2)
                cv2.putText(frame_resized, str(det.track_id), (int(det.bb_left // 2), int(det.bb_top // 2)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        # Create the display canvas and add elements
        canvas = np.ones((canvas_height, canvas_width, 3), dtype=np.uint8) * 255
        canvas[0:video_display_height, 0:video_display_width] = frame_resized

        # Add trajectory graph
        trajectory_img = update_trajectory_graph()
        traj_height, traj_width, _ = trajectory_img.shape
        resized_traj_img = cv2.resize(trajectory_img, (390, int(traj_height * (390 / traj_width))))
        canvas[10:10+resized_traj_img.shape[0], video_display_width+10:video_display_width+10+resized_traj_img.shape[1]] = resized_traj_img

        # Display car count
        cv2.putText(canvas, f"Cars: {len(unique_car_ids)}", car_count_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

        # Show the final canvas
        cv2.imshow("Real-Time Tracking and Analysis", canvas)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_id += 1

    cap.release()
    cv2.destroyAllWindows()
    plt.close()

# # Main function to run video and plot real-time trajectories
# def main(args):
#     yolo_version = "yolov8x"
#     detector = Detector()
#     detector.load(args.cam_para, yolo_version)
#     tracker = UCMCTrack(args.a, args.a, args.wx, args.wy, args.vmax, args.cdt, args.fps, "MOT", args.high_score, False, None)
#     unique_car_ids = set()
#     trajectory_data = defaultdict(lambda: {'coords': [], 'class_id': None})

#     cap = cv2.VideoCapture(args.video)
#     fps = cap.get(cv2.CAP_PROP_FPS)
#     width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#     height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

#     # Define scaled down dimensions for the video frame
#     video_display_width = width // 2  # Adjust this value if needed
#     video_display_height = height // 2
#     canvas_width = video_display_width + 400  # Extra space for trajectory plot
#     canvas_height = video_display_height + 10#+ 200  # Extra space for car count display

#     # Set up position for car count display
#     car_count_position = (video_display_width + 10, canvas_height - 20)  # Bottom-right corner

#     # Set up Matplotlib figure for the trajectory plot
#     fig, ax = plt.subplots()
#     plt.ion()  # Interactive mode for real-time plotting

#     def update_trajectory_graph():
#         ax.clear()
#         ax.set_title("Trajectories of Tracked Vehicles")
#         ax.set_xlabel("X Coordinate")
#         ax.set_ylabel("Y Coordinate")
#         if not trajectory_data:
#             print("No trajectory data to display.")
#         for track_id, data in trajectory_data.items():
#             if data['coords']:
#                 x_vals, y_vals = zip(*data['coords'])
#                 class_id = data['class_id']
#                 label = "Car" if class_id == 2 else f"Class {class_id}"
#                 ax.plot(x_vals, y_vals, marker="o", label=f'Track {track_id} ({label})')
#         if trajectory_data:
#             ax.legend(loc="upper right")
#         fig.canvas.draw()
#         fig_image = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
#         fig_image = fig_image.reshape(fig.canvas.get_width_height()[::-1] + (4,))
#         fig_image = cv2.cvtColor(fig_image, cv2.COLOR_RGBA2RGB)  # Convert RGBA to RGB
#         return fig_image

#     frame_id = 1
#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             print("No more frames to read.")
#             break

#         # Scale down the video frame for display
#         frame_resized = cv2.resize(frame, (video_display_width, video_display_height))

#         dets = detector.get_dets(frame, args.conf_thresh, [2, 5, 7])
#         tracker.update(dets, frame_id)

#         # Update the unique car count and trajectory data
#         for det in dets:
#             if det.track_id > 0:
#                 unique_car_ids.add(det.track_id)
#                 trajectory_data[det.track_id]['coords'].append((det.y[0, 0], det.y[1, 0]))
#                 trajectory_data[det.track_id]['class_id'] = det.det_class
#                 cv2.rectangle(frame_resized, (int(det.bb_left // 2), int(det.bb_top // 2)),
#                               (int((det.bb_left + det.bb_width) // 2), int((det.bb_top + det.bb_height) // 2)),
#                               (0, 255, 0), 2)
#                 cv2.putText(frame_resized, str(det.track_id), (int(det.bb_left // 2), int(det.bb_top // 2)),
#                             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

#         # Create a white canvas and place the video frame on it
#         canvas = np.ones((canvas_height, canvas_width, 3), dtype=np.uint8) * 255
#         canvas[0:video_display_height, 0:video_display_width] = frame_resized

#         # Get the updated trajectory plot and place it in the top-right corner
#         trajectory_img = update_trajectory_graph()
#         traj_height, traj_width, _ = trajectory_img.shape
#         resized_traj_img = cv2.resize(trajectory_img, (390, int(traj_height * (390 / traj_width))))  # Resize proportionally
#         canvas[10:10+resized_traj_img.shape[0], video_display_width+10:video_display_width+10+resized_traj_img.shape[1]] = resized_traj_img

#         # Display car count in the bottom-right corner
#         cv2.putText(canvas, f"Cars: {len(unique_car_ids)}", car_count_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

#         # Display the canvas
#         cv2.imshow("Real-Time Tracking and Analysis", canvas)

#         # Quit if 'q' is pressed
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

#         frame_id += 1
#         # Debugging output
#         print(f"Frame {frame_id}, Cars Count: {len(unique_car_ids)}, Trajectories: {len(trajectory_data)}")

#     cap.release()
#     cv2.destroyAllWindows()
#     plt.close()

# Argument parser setup
parser = argparse.ArgumentParser(description='Real-time Tracking and Trajectory Plotting')
parser.add_argument('--video', type=str, default="C:/research/new30s.mp4", help='video file name')
parser.add_argument('--cam_para', type=str, default="C:/research/UCMCTrack/demo/cam_para.txt", help='camera parameter file name')
parser.add_argument('--wx', type=float, default=5, help='wx')
parser.add_argument('--wy', type=float, default=5, help='wy')
parser.add_argument('--vmax', type=float, default=10, help='vmax')
parser.add_argument('--a', type=float, default=150.0, help='assignment threshold')
parser.add_argument('--cdt', type=float, default=15.0, help='coasted deletion time')
parser.add_argument('--high_score', type=float, default=0.4, help='high score threshold')
parser.add_argument('--conf_thresh', type=float, default=0.001, help='detection confidence threshold')
parser.add_argument('--fps', type=float, default=30, help='frames per second')
args = parser.parse_args()

# Run the main function
main(args)


In [ ]:

        # for det in dets:
        #     center = (int(det.bb_left + det.bb_width / 2), int(det.bb_top + det.bb_height / 2))
        #     if cv2.pointPolygonTest(roi_points, center, False) >= 0:
        #         unique_car_ids.add(det.track_id)
        #         trajectory_data[det.track_id]['coords'].append((det.y[0, 0], det.y[1, 0]))
        #         class_name = class_labels.get(det.det_class, "Unknown")
        #         label = f"{class_name} {det.conf:.2f}"

        #         # Draw the bounding box, classification, and confidence score
        #         cv2.rectangle(
        #             frame_resized,
        #             (int(det.bb_left // 2), int(det.bb_top // 2)),
        #             (int((det.bb_left + det.bb_width) // 2), int((det.bb_top + det.bb_height) // 2)),
        #             (0, 255, 0), 2
        #         )
        #         cv2.putText(
        #             frame_resized, label,
        #             (int(det.bb_left // 2), int((det.bb_top // 2) - 10)),
        #             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1
        #         )




# # Main function
# def main(args):
#     yolo_version = "yolo11x"
#     detector = Detector()
#     detector.load(args.cam_para, yolo_version)
#     tracker = UCMCTrack(args.a, args.a, args.wx, args.wy, args.vmax, args.cdt, args.fps, "MOT", args.high_score, False, None)
#     unique_car_ids = set()
#     trajectory_data = defaultdict(lambda: {'coords': [], 'class_id': None})

#     # Define class labels
#     class_labels = {2: "Car", 5: "Bus", 7: "Truck"}  # Define labels for known classes

#     cap = cv2.VideoCapture(args.video)
#     fps = cap.get(cv2.CAP_PROP_FPS)
#     width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#     height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

#     video_display_width = width // 2
#     video_display_height = height // 2
#     canvas_width = video_display_width + 400
#     canvas_height = video_display_height + 10
#     car_count_position = (video_display_width + 10, canvas_height - 20)

#     fig, ax = plt.subplots()
#     plt.ion()

#     def update_trajectory_graph():
#         ax.clear()
#         ax.set_title("Trajectories of Tracked Vehicles")
#         ax.set_xlabel("X Coordinate")
#         ax.set_ylabel("Y Coordinate")
#         for track_id, data in trajectory_data.items():
#             if data['coords']:
#                 x_vals, y_vals = zip(*data['coords'])
#                 ax.plot(x_vals, y_vals, marker="o", label=f'Track {track_id}')
#         fig.canvas.draw()
#         fig_image = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
#         fig_image = fig_image.reshape(fig.canvas.get_width_height()[::-1] + (4,))
#         fig_image = cv2.cvtColor(fig_image, cv2.COLOR_RGBA2RGB)
#         return fig_image

#     frame_id = 1
#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             break

#         frame_resized = cv2.resize(frame, (video_display_width, video_display_height))
#         original_size = (width, height)
#         resized_size = (video_display_width, video_display_height)
#         frame_resized = apply_green_roi_mask(frame_resized, roi_points, original_size, resized_size)

#         dets = detector.get_dets(frame, args.conf_thresh, [2, 5, 7])
#         tracker.update(dets, frame_id)

#         for det in dets:
#             center = (int(det.bb_left + det.bb_width / 2), int(det.bb_top + det.bb_height / 2))
#             if cv2.pointPolygonTest(roi_points, center, False) >= 0:
#                 unique_car_ids.add(det.track_id)
#                 trajectory_data[det.track_id]['coords'].append((det.y[0, 0], det.y[1, 0]))

#                 # Define the class name, tracking ID, and confidence score
#                 class_name = class_labels.get(det.det_class, "Unknown")
#                 label = f"{class_name} ID:{det.track_id} {det.conf:.2f}"

#                 # Draw the bounding box, classification, confidence score, and tracking ID
#                 cv2.rectangle(
#                     frame_resized,
#                     (int(det.bb_left // 2), int(det.bb_top // 2)),
#                     (int((det.bb_left + det.bb_width) // 2), int((det.bb_top + det.bb_height) // 2)),
#                     (0, 255, 0), 2
#                 )
#                 # Display the label with classification, confidence, and ID above the bounding box
#                 cv2.putText(
#                     frame_resized, label,
#                     (int(det.bb_left // 2), int((det.bb_top // 2) - 10)),
#                     cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1
#                 )

#         canvas = np.ones((canvas_height, canvas_width, 3), dtype=np.uint8) * 255
#         canvas[0:video_display_height, 0:video_display_width] = frame_resized

#         trajectory_img = update_trajectory_graph()
#         traj_height, traj_width, _ = trajectory_img.shape
#         resized_traj_img = cv2.resize(trajectory_img, (390, int(traj_height * (390 / traj_width))))
#         canvas[10:10+resized_traj_img.shape[0], video_display_width+10:video_display_width+10+resized_traj_img.shape[1]] = resized_traj_img

#         cv2.putText(canvas, f"Cars: {len(unique_car_ids)}", car_count_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
#         cv2.imshow("Real-Time Tracking and Analysis", canvas)

#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

#         frame_id += 1

#     cap.release()
#     cv2.destroyAllWindows()
#     plt.close()


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO
from tracker.ucmc import UCMCTrack
from detector.mapper import Mapper
from collections import defaultdict
import argparse
# Define the ROI points as a numpy array
roi_points = np.array([
    [28, 1046], [109, 977], [269, 846], [426, 715], [507, 614],
    [570, 520], [521, 456], [427, 411], [308, 392], [182, 372],
    [39, 350], [196, 251], [338, 273], [518, 299], [663, 315],
    [805, 319], [920, 292], [1045, 236], [1099, 195], [1155, 154],
    [1243, 103], [1311, 114], [1256, 176], [1205, 249], [1198, 346],
    [1286, 433], [1454, 500], [1634, 555], [1786, 595], [1902, 621],
    [1888, 797], [1665, 738], [1408, 684], [1184, 650], [1012, 680],
    [856, 774], [739, 921], [659, 1047]
], np.int32)

# Define Detection class
class Detection:
    def __init__(self, id, bb_left=0, bb_top=0, bb_width=0, bb_height=0, conf=0, det_class=0):
        self.id = id
        self.bb_left = bb_left
        self.bb_top = bb_top
        self.bb_width = bb_width
        self.bb_height = bb_height
        self.conf = conf
        self.det_class = det_class
        self.track_id = 0
        self.y = np.zeros((2, 1))
        self.R = np.eye(4)

# Define Detector class
class Detector:
    def __init__(self):
        self.seq_length = 0
        self.gmc = None

    def load(self, cam_para_file, yolo_version):
        self.mapper = Mapper(cam_para_file, "MOT17")
        self.model = YOLO(f'pretrained/{yolo_version}.pt')

    def get_dets(self, img, conf_thresh=0, det_classes=[0]):
        dets = []
        frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = self.model(frame, imgsz=1088)
        det_id = 0
        for box in results[0].boxes:
            conf = box.conf.cpu().numpy()[0]
            bbox = box.xyxy.cpu().numpy()[0]
            cls_id = box.cls.cpu().numpy()[0]
            w = bbox[2] - bbox[0]
            h = bbox[3] - bbox[1]
            if w <= 10 and h <= 10 or cls_id not in det_classes or conf <= conf_thresh:
                continue

            det = Detection(det_id)
            det.bb_left = bbox[0]
            det.bb_top = bbox[1]
            det.bb_width = w
            det.bb_height = h
            det.conf = conf
            det.det_class = cls_id
            det.y, det.R = self.mapper.mapto([det.bb_left, det.bb_top, det.bb_width, det.bb_height])
            det_id += 1
            dets.append(det)

        return dets
    
def apply_green_roi_mask(frame, points, original_size, resized_size):
    # Calculate the scaling factors for width and height
    scale_x = resized_size[0] / original_size[0]
    scale_y = resized_size[1] / original_size[1]

    # Scale the ROI points according to the new size
    scaled_points = np.array([[int(x * scale_x), int(y * scale_y)] for x, y in points], np.int32)

    # Create overlay with the ROI in green
    overlay = frame.copy()
    cv2.fillPoly(overlay, [scaled_points], (0, 255, 0))  # Green color for ROI

    # Apply transparency to the overlay
    alpha = 0.3  # Transparency level
    cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
    return frame

def main(args):
    yolo_version = "yolov8x"
    detector = Detector()
    detector.load(args.cam_para, yolo_version)
    tracker = UCMCTrack(args.a, args.a, args.wx, args.wy, args.vmax, args.cdt, args.fps, "MOT", args.high_score, False, None)
    unique_car_ids = set()
    trajectory_data = defaultdict(lambda: {'coords': [], 'class_id': None})

    cap = cv2.VideoCapture(args.video)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define scaled down dimensions for the video frame
    video_display_width = width // 2
    video_display_height = height // 2
    canvas_width = video_display_width + 400
    canvas_height = video_display_height + 10
    car_count_position = (video_display_width + 10, canvas_height - 20)

    fig, ax = plt.subplots()
    plt.ion()

    def update_trajectory_graph():
        ax.clear()
        ax.set_title("Trajectories of Tracked Vehicles")
        ax.set_xlabel("X Coordinate")
        ax.set_ylabel("Y Coordinate")
        for track_id, data in trajectory_data.items():
            if data['coords']:
                x_vals, y_vals = zip(*data['coords'])
                ax.plot(x_vals, y_vals, marker="o", label=f'Track {track_id}')
        fig.canvas.draw()
        fig_image = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
        fig_image = fig_image.reshape(fig.canvas.get_width_height()[::-1] + (4,))
        fig_image = cv2.cvtColor(fig_image, cv2.COLOR_RGBA2RGB)
        return fig_image

    frame_id = 1
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Apply the green ROI mask to the video frame
        frame_resized = cv2.resize(frame, (video_display_width, video_display_height))
        # Original size of the video
        original_size = (width, height)

        # New resized dimensions for display
        resized_size = (video_display_width, video_display_height)

        # Apply the scaled ROI mask to the resized frame
        frame_resized = apply_green_roi_mask(frame_resized, roi_points, original_size, resized_size)

        # Process detections and filter within the ROI
        dets = detector.get_dets(frame, args.conf_thresh, [2, 5, 7])
        tracker.update(dets, frame_id)

        for det in dets:
            # Check if detection is inside the ROI
            center = (int(det.bb_left + det.bb_width / 2), int(det.bb_top + det.bb_height / 2))
            if cv2.pointPolygonTest(roi_points, center, False) >= 0:
                unique_car_ids.add(det.track_id)
                trajectory_data[det.track_id]['coords'].append((det.y[0, 0], det.y[1, 0]))
                cv2.rectangle(frame_resized, (int(det.bb_left // 2), int(det.bb_top // 2)),
                              (int((det.bb_left + det.bb_width) // 2), int((det.bb_top + det.bb_height) // 2)),
                              (0, 255, 0), 2)
                cv2.putText(frame_resized, str(det.track_id), (int(det.bb_left // 2), int(det.bb_top // 2)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        # Create the display canvas and add elements
        canvas = np.ones((canvas_height, canvas_width, 3), dtype=np.uint8) * 255
        canvas[0:video_display_height, 0:video_display_width] = frame_resized

        # Add trajectory graph
        trajectory_img = update_trajectory_graph()
        traj_height, traj_width, _ = trajectory_img.shape
        resized_traj_img = cv2.resize(trajectory_img, (390, int(traj_height * (390 / traj_width))))
        canvas[10:10+resized_traj_img.shape[0], video_display_width+10:video_display_width+10+resized_traj_img.shape[1]] = resized_traj_img

        # Display car count
        cv2.putText(canvas, f"Cars: {len(unique_car_ids)}", car_count_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

        # Show the final canvas
        cv2.imshow("Real-Time Tracking and Analysis", canvas)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_id += 1

    cap.release()
    cv2.destroyAllWindows()
    plt.close()



# Argument parser setup
# parser = argparse.ArgumentParser(description='Real-time Tracking and Trajectory Plotting')
# parser.add_argument('--video', type=str, default="C:/research/new30s.mp4", help='video file name')
# parser.add_argument('--cam_para', type=str, default="C:/research/UCMCTrack/demo/cam_para.txt", help='camera parameter file name')
# parser.add_argument('--wx', type=float, default=5, help='wx')
# parser.add_argument('--wy', type=float, default=5, help='wy')
# parser.add_argument('--vmax', type=float, default=10, help='vmax')
# parser.add_argument('--a', type=float, default=150.0, help='assignment threshold')
# parser.add_argument('--cdt', type=float, default=15.0, help='coasted deletion time')
# parser.add_argument('--high_score', type=float, default=0.4, help='high score threshold')
# parser.add_argument('--conf_thresh', type=float, default=0.0001, help='detection confidence threshold')
# parser.add_argument('--fps', type=float, default=30, help='frames per second')
# args = parser.parse_args()

parser = argparse.ArgumentParser(description='Real-time Tracking and Trajectory Plotting')
parser.add_argument('--video', type=str, default="C:/research/new30s.mp4", help='video file name')
parser.add_argument('--cam_para', type=str, default="C:/research/UCMCTrack/demo/cam_para.txt", help='camera parameter file name')
parser.add_argument('--wx', type=float, default=10, help='wx')
parser.add_argument('--wy', type=float, default=10, help='wy')
parser.add_argument('--vmax', type=float, default=12, help='vmax')
parser.add_argument('--a', type=float, default=100.0, help='assignment threshold')
parser.add_argument('--cdt', type=float, default=10.0, help='coasted deletion time')
parser.add_argument('--high_score', type=float, default=0.001, help='high score threshold')
parser.add_argument('--conf_thresh', type=float, default=0.5, help='detection confidence threshold')
parser.add_argument('--fps', type=float, default=30, help='frames per second')
args = parser.parse_args()

# Run the main function
main(args)